# Load Models

In [ ]:
!gdown --id 1Nrj3gi-ENBwsEqjsGB68pWGrQI8fFWC6

Downloading...
From: https://drive.google.com/uc?id=1Nrj3gi-ENBwsEqjsGB68pWGrQI8fFWC6
To: /content/DEMO.zip
100% 213M/213M [00:03<00:00, 66.0MB/s]


In [ ]:
!unzip DEMO.zip

Archive:  DEMO.zip
   creating: DEMO/
  inflating: DEMO/fake_news_of_train.pkl  
  inflating: DEMO/kmeans_15.pkl      
  inflating: DEMO/kmeans_14.pkl      
  inflating: DEMO/kmeans_10.pkl      
  inflating: DEMO/kmeans_11.pkl      
  inflating: DEMO/kmeans_12.pkl      
  inflating: DEMO/kmeans_13.pkl      
  inflating: DEMO/Encoder_Bi-Vae-Density.zip  


In [ ]:
!unzip DEMO/Encoder_Bi-Vae-Density.zip

Archive:  DEMO/Encoder_Bi-Vae-Density.zip
   creating: Encoder_Bi-Vae-Density/
  inflating: Encoder_Bi-Vae-Density/saved_model.pb  
  inflating: Encoder_Bi-Vae-Density/keras_metadata.pb  
   creating: Encoder_Bi-Vae-Density/assets/
   creating: Encoder_Bi-Vae-Density/variables/
  inflating: Encoder_Bi-Vae-Density/variables/variables.index  
  inflating: Encoder_Bi-Vae-Density/variables/variables.data-00000-of-00001  


## Load Bi-VAE-Density

In [ ]:
import tensorflow as tf

bi_vae_density = tf.keras.models.load_model('Encoder_Bi-Vae-Density')

## Load Kmeans

In [ ]:
import pickle

l_kmeans = []

for i in range(6):
  with open('DEMO/kmeans_1' + str(i) + '.pkl', 'rb') as file:
    l_kmeans.append(pickle.load(file))

## Load fake news of train

In [ ]:
import pandas as pd
import numpy as np
df_fk_train = pd.read_pickle('DEMO/fake_news_of_train.pkl')
df_train = np.array(df_fk_train['DistilBERT Multilingua'].to_list())

# DBERTML Embeddings (USE GPU)

In [ ]:
!pip install sentence-transformers==1.0.4 #version used in the fake news collections

     |████████████████████████████████| 74 kB 2.4 MB/s 
     |████████████████████████████████| 3.1 MB 10.9 MB/s 
     |████████████████████████████████| 1.2 MB 41.9 MB/s 
     |████████████████████████████████| 596 kB 55.8 MB/s 
     |████████████████████████████████| 59 kB 7.0 MB/s 
     |████████████████████████████████| 3.3 MB 43.3 MB/s 
     |████████████████████████████████| 895 kB 77.3 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-1.0.4-py3-none-any.whl size=114306 sha256=78750ed68821ab467e0d2be3b70b511b635008099ea444c8214422c7077ab808
  Stored in directory: /root/.cache/pip/wheels/36/29/9e/1d73f2751adb27f61c414d2656cef08c8353b9ad7cb7f149ef
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from sentence_transformers import SentenceTransformer

In [ ]:
def sentence_embedding(txts):

  model = SentenceTransformer('distiluse-base-multilingual-cased')

  sentences =[]

  for txt in txts:
    txt.replace('\\\\t', ' ')
    txt.replace('\\\\r', ' ')
    txt.replace('\\\\n',' ')
    txt.replace('\\t', ' ')
    txt.replace('\\r', ' ')
    txt.replace('\\n',' ')
    txt.replace('\t', ' ')
    txt.replace('\r', ' ')
    txt.replace('\n',' ')
    sentences.append(txt)

  sentence_embeddings = model.encode(sentences)

  return sentence_embeddings 

# Density Information

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score
import numpy as np 

def return_density_inf(l_kmeans, df_train, df_new):
    l_x_new = []

    len_train = len(df_train)
    len_new = len(df_new)

    for kmeans in l_kmeans:
        x_new_temp = []

        dfs = np.concatenate([df_train, df_new])
        labels = np.concatenate([kmeans.labels_, kmeans.predict(df_new.astype('float64'))])
          
        silho = silhouette_samples(dfs, labels).reshape(len_train + len_new, 1)
        l_x_new.append(silho[len_train:])

    return np.concatenate(l_x_new, axis=1)

# DEMO

In [ ]:
inputs = [
          'this is a real news',
          'essa é uma noticia falsa sobre o bolsonaro',
          'essa aqui é outra noticia falsa que fala sobre politica'
]

In [ ]:
embeddings_DBERTML = sentence_embedding(inputs)

  0%|          | 0.00/504M [00:00<?, ?B/s]

In [ ]:
densities = return_density_inf(l_kmeans, df_train, np.array(embeddings_DBERTML))

In [ ]:
embeddings_BiVae, _, _ = bi_vae_density.predict([embeddings_DBERTML,densities]) 